## INFERENCE MODEL
Setup inference model

In [1]:
import os, sys
sys.path.append(os.path.split(sys.path[0])[0])

#Dependencies
import dill
import h5py
import math
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sbi.inference import SNPE, prepare_for_sbi, simulate_for_sbi
import sbi.utils as utils
from sbi.utils.get_nn_models import posterior_nn  # For SNLE: likelihood_nn(). For SNRE: classifier_nn()
from scipy import signal, stats as spstats
import scipy.io
import torch
import torch.nn as nn
import torch.nn.functional as F

#Project Imports
import config.params as params
import config.paths as paths
from stylized_module.base.passive_model_soma_injection import run_pm_simulation
from stylized_module.models.cnn import SummaryNet
from utils.metrics.corrcoef import corrcoef, max_corrcoef
from utils.plotting.plot_results import plot_LFP_traces,plot_LFP_heatmap
from utils.spike_window import first_pk_tr, get_spike_window

Helper Functions

In [2]:
rng = np.random.default_rng(123412)

def model(param,whole_trace=False):
    #Replace theta with random number
    theta = [rng.uniform(low=params.IM_THETA_BOUNDS[0], high=params.IM_THETA_BOUNDS[1])]
    sim.set_loc_param(torch.cat((param[:2], torch.tensor(theta), param[2:4])))
    scalVal = 1 #10 ** param[5]
    sim.set_scale(scalVal)
    sim.set_geo_param(param[4:])
    sim.create_cells()
    sim.run()
    lfp = sim.get_lfp().T
    filtered_lfp = signal.lfilter(filt_b,filt_a,lfp,axis=0) # filter along row of the lfp 2d-array, if each row is a channel
    if not whole_trace:
        start,end = get_spike_window(filtered_lfp,win_size=params.PM_WINDOW_SIZE,align_at=fst_idx)
        filtered_lfp = filtered_lfp[start:end,:]
    return filtered_lfp


def Stats(lfp):
    """
    Calculates summary statistics
    results = model(params)
    """
    lfp = np.asarray(lfp)
    
    avg = np.mean(lfp,axis=0) # average voltage of each channel
#     stdDev = np.std(lfp,axis=0) # stDev of the voltage of each channel
    tT = np.argmin(lfp,axis=0)
    tP = np.argmax(lfp,axis=0)
    Troughs = np.take_along_axis(lfp,np.expand_dims(tT,axis=0),axis=0)
    Peaks = np.take_along_axis(lfp,np.expand_dims(tP,axis=0),axis=0)
    relT = tP-tT
    stats_list = [avg,Troughs,Peaks,relT]
    
    def statscalc(stats):
        stats = stats.ravel()
        mean = np.mean(stats)
        std = np.std(stats)
        m = np.argmin(stats)
        min_pos = params.IM_Y_DISTANCE[m]
        min_val = stats[m]
        M = np.argmax(stats)
        max_pos = params.IM_Y_DISTANCE[M] 
        max_val = stats[M]
        All = np.array([mean,std,min_pos,min_val,max_pos,max_val])
        return All
    
    allStats = np.concatenate([statscalc(x) for x in stats_list])
    return allStats

def cat_output(lfp):
    output = np.concatenate((lfp.ravel(),Stats(lfp)))
    return torch.from_numpy(output)

def simulation(params):
    lfp = model(params)
    return cat_output(lfp)

In [3]:
filt_b,filt_a = signal.butter(params.IM_BUTTERWORTH_ORDER,
                              params.IM_CRITICAL_FREQUENCY,
                              params.IM_BANDFILTER_TYPE,
                              fs=params.IM_FILTER_SAMPLING_RATE)

sim, window_size, x0_trace, t0 = run_pm_simulation()

fst_idx = first_pk_tr(x0_trace)

In [4]:
simulator, prior = prepare_for_sbi(simulation, params.IM_PRIOR_DISTRIBUTION)
x_o = cat_output(x0_trace)

In [5]:
# instantiate the neural density estimator
density_estimator_build_fun = posterior_nn(model=params.IM_POSTERIOR_MODEL_ESTIMATOR,
                                           embedding_net=params.IM_EMBEDDED_NETWORK,
                                           hidden_features=params.IM_POSTERIOR_MODEL_HIDDEN_LAYERS)

inference = SNPE(prior=prior,density_estimator=density_estimator_build_fun,show_progress_bars=True)

In [6]:
posteriors = []
proposal = prior

for i in range(params.IM_NUMBER_OF_ROUNDS):
    theta, x = simulate_for_sbi(simulator,proposal,num_simulations=params.IM_NUMBER_OF_SIMULATIONS)
     # In `SNLE` and `SNRE`, you should not pass the `proposal` to `.append_simulations()`
    density_estimator = inference.append_simulations(theta, x, proposal=proposal).train()
    posterior = inference.build_posterior(density_estimator,sample_with_mcmc=True) # sample_with_mcmc=True
    
    with open(paths.POSTERIOR_SAVE + str(i) + ".pkl", "wb") as handle:
        pickle.dump(posterior, handle)
    
#     with open(paths.INFERENCER_SAVE + str(i) + ".pkl", "wb") as handle:
#         dill.dump(inference, handle)
        
    posteriors.append(posterior)
    proposal = posterior.set_default_x(x_o)

Running 5000 simulations.:   0%|          | 0/5000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
samples = posterior.sample((1000,), x=x_o, sample_with_mcmc=True) #, sample_with_mcmc=True

#posterior.leakage_correction(x_o, num_rejection_samples=1000)
log_probability = posterior.log_prob(samples,x=x_o, norm_posterior=False) #, norm_posterior=False
plt.hist(log_probability.numpy())

In [ ]:
sample_idx = np.argmax(log_probability)
predicted_post = samples[sample_idx]
predicted_post

In [ ]:
save_xo = params.IM_SAVE_X0
# save_xo = 'x_0_traces.pdf'
fig,ax = plot_LFP_traces(t0,x0_trace,savefig=params.IM_SAVE_X0)
# save_xo = 'x_0_HTmap.pdf'
elec_idx = slice(30,-10)
fig,ax = plot_LFP_heatmap(t0,params.IM_Y_DISTANCE[elec_idx],x0_trace[:,elec_idx],vlim=[-1,1],savefig=params.IM_SAVE_X0)

In [ ]:
# predicted_lfp = model(params.IM_RANDOM_SAMPLE,whole_trace=True)
predicted_lfp = model(predicted_post,whole_trace=True)

savefig = None
fig,ax = plot_LFP_traces(sim.t(),predicted_lfp)
start,end = get_spike_window(predicted_lfp,win_size=params.PM_WINDOW_SIZE,align_at=fst_idx)
predicted_lfp_win = predicted_lfp[start:end,:]

fig,ax = plot_LFP_traces(t0,
                         predicted_lfp_win,
                         savefig=paths.PASSIVE_INFERENCE_SAVE_TRACES)

fig,ax = plot_LFP_heatmap(t0,
                          params.IM_Y_DISTANCE[elec_idx],
                          predicted_lfp_win[:,elec_idx],
                          vlim=[-1,1],
                          savefig=paths.PASSIVE_INFERENCE_SAVE_HEATMAPS)

In [ ]:
max_corr, max_ind = max_corrcoef(x0_trace,predicted_lfp)
print(max_corr,max_ind)
max_corr, max_ind = max_corrcoef(x0_trace,predicted_lfp_win)
print(max_corr,max_ind)

In [ ]:
hf = h5py.File(paths.PASSIVE_INFERENCE_RESULTS_DATA, 'w')
hf.create_dataset('LFP',data=predicted_lfp)
hf.create_dataset('samples',data=samples.numpy())
hf.create_dataset('log_prob',data=log_probability.numpy())
hf.close()

scipy.io.savemat(paths.PASSIVE_INFERENCE_RESULTS_MATLAB_DATA,
                 {'LFP':predicted_lfp_win[:,elec_idx],'t':t0,'y_dist':params.IM_Y_DISTANCE[elec_idx]})

scipy.io.savemat(paths.PASSIVE_INFERENCE_RESULTS_X0_MATLAB_DATA,
                 {'LFP':x0_trace[:,elec_idx],'t':t0,'y_dist':params.IM_Y_DISTANCE[elec_idx]})

In [ ]:
labels_params = params.IM_GRAPHING_LABELS
fig, axes = utils.pairplot(samples,
                           limits = params.IM_PARAMETER_BOUNDS,
                           ticks = params.IM_PARAMETER_BOUNDS,
                           fig_size=(12,12),
                           #points=true_params,
                           points_offdiag={'markersize': 6},
                           labels = params.IM_GRAPHING_LABELS,
                           points_colors='r');

In [ ]:
%matplotlib inline

labels_params = params.IM_GRAPHING_LABELS
bounds_ticks = params.IM_PARAMETER_BOUNDS
bounds_ticks[2] = bounds_ticks[4] = [0,3.14]
fig, axes = utils.pairplot(samples,
                           limits = params.IM_PARAMETER_BOUNDS,
                           ticks = params.IM_PARAMETER_BOUNDS,
                           fig_size=(15,15),
                           #points=true_params,
                           points_offdiag={'markersize': 6},
                           labels = params.IM_GRAPHING_LABELS,
                           points_colors='r');

# axes[4][4].set_xlabel('\u03BB',fontsize = 40)
for i in range(len(params.IM_PARAMETER_BOUNDS)):
    axes[i][i].tick_params('x',labelsize=15)
    axes[i][i].xaxis.label.set_fontsize(40)

for i,b in enumerate(params.IM_PARAMETER_BOUNDS):
    axes[i][i].set_xticklabels([str(b[0]),str(b[1])],fontsize=30)

plt.savefig(paths.PASSIVE_INFERENCE_SAVE_KDE,bbox_inches='tight',transparent=True)